In [2]:
from pymongo import MongoClient
from pprint import pprint
import random

In [3]:
client = MongoClient('mongodb://localhost:27017')
db=client.dat
coll = db.crossref

In [4]:
def random_doc(max_ix=100000):
    ix = random.randrange(max_ix)
    pprint(coll.find().limit(-1).skip(ix).next())
    
def query(q, lim=1):
    pprint(coll.find_one(q))

In [4]:
for x in range(10):
    random_doc()
    print '\n\n\n'

{u'DOI': u'10.1001/archderm.1931.01450011144017',
 u'ISSN': [u'0003-987X'],
 u'URL': u'http://dx.doi.org/10.1001/archderm.1931.01450011144017',
 u'_id': ObjectId('58d1a9a00c62134f84674b39'),
 u'author': [{u'affiliation': [],
              u'family': u'Fraser',
              u'given': u'J. Frank'}],
 u'cited-count': 0,
 u'citing-count': 0,
 u'container-title': [u'Archives of Dermatology'],
 u'content-domain': {u'crossmark-restriction': False, u'domain': []},
 u'created': {u'date-parts': [[2011, 11, 28]],
              u'date-time': u'2011-11-28T21:46:17Z',
              u'timestamp': 1322516777000L},
 u'deposited': {u'date-parts': [[2016, 12, 17]],
                u'date-time': u'2016-12-17T01:03:44Z',
                u'timestamp': 1481936624000L},
 u'indexed': {u'date-parts': [[2016, 12, 19]],
              u'date-time': u'2016-12-19T20:23:33Z',
              u'timestamp': 1482179013323L},
 u'issue': u'6',
 u'issued': {u'date-parts': [[1931, 12, 1]]},
 u'member': u'http://id.crossref.o

In [16]:
query({"type" : 'book-chapter'})

{u'DOI': u'10.1002/0470007257.ch1',
 u'ISBN': [u'http://id.crossref.org/isbn/9780470007259',
           u'http://id.crossref.org/isbn/9780471684008'],
 u'URL': u'http://dx.doi.org/10.1002/0470007257.ch1',
 u'_id': ObjectId('58d1bd080c62134f847f1950'),
 u'cited-count': 0,
 u'citing-count': 0,
 u'container-title': [u'Hazardous Waste Operations and Emergency Response Manual'],
 u'content-domain': {u'crossmark-restriction': False, u'domain': []},
 u'created': {u'date-parts': [[2006, 1, 26]],
              u'date-time': u'2006-01-26T23:46:50Z',
              u'timestamp': 1138319210000L},
 u'deposited': {u'date-parts': [[2013, 7, 7]],
                u'date-time': u'2013-07-07T02:22:44Z',
                u'timestamp': 1373163764000L},
 u'indexed': {u'date-parts': [[2016, 11, 8]],
              u'date-time': u'2016-11-08T12:01:40Z',
              u'timestamp': 1478606500057L},
 u'issued': {u'date-parts': [[2006, 1, 26]]},
 u'member': u'http://id.crossref.org/member/311',
 u'original-title': 

In [ ]:
all_types = []
result = coll.find({}, {'type': 1}).limit(100)
for item in result:
    all_types.append(result.next()['type'])
    
